In [1]:
import sys
sys.path.append('../src')

import glosat_utils
from glosat_utils import *
import matplotlib.pyplot as plt 
import pickle
import bs4 as bs


[2022-11-17 12:15:34] Model res_unet18 initialisated with norm_layer=InstanceNorm2d({'momentum': 0.1, 'affine': True, 'track_running_stats': False}) and kwargs {'encoder_name': 'resnet18', 'reduced_layers': False, 'no_maxpool': False, 'conv_as_maxpool': True, 'use_deconv': True, 'use_upcatconv': False, 'use_conv1x1': False, 'pretrained_encoder': False}


In [2]:
import cv2
import cv2 as cv
import imutils
import os
from os import path
import numpy as np
import torch
import matplotlib.pyplot as plt
import pickle

# Add code to sys.path
import matplotlib.pyplot as plt 

from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
import cv2 as cv
import collections

import dla.src.table_structure_analysis as tsa
import dla.src.xml_utils as xml_utils
from dla.src.image_utils import put_box, put_line
import pytesseract
import numpy as np

from statistics import mean

import importlib
import glosat_utils

from glosat_utils import *

import bs4 as bs
from PIL import Image

In [3]:
from bs4 import BeautifulSoup, Tag

In [4]:
try:
    import xml_utils as xml_utils
except:
    import dla.src.xml_utils as xml_utils

import argparse
import os

In [5]:
import copy 

In [6]:
def get_coordinates_transkribus(cordinates):
    imagedraw=[]
    points=cordinates.split()
    x_pt = []
    y_pt = []
    for cord in points:
        (x,y)=cord.split(',')
        x_pt.append(int(x))
        y_pt.append(int(y))
        
    imagedraw.append(min(x_pt))
    imagedraw.append(min(y_pt))
    imagedraw.append(max(x_pt))
    imagedraw.append(max(y_pt))
    return(imagedraw)

In [7]:
datapath = '/data/glosat/Code-Git/docExtractor-master/glosat/Data_conversion/Fine'
transkribus = f'{datapath}/Transkribus'
icdar = f'{datapath}/ICDAR'
output_path = f'{datapath}/ICDAR_Transkribus'

In [34]:
filename = '4.xml'

### Read Transkribus XML file

In [35]:
f = open(f'{transkribus}/{filename}', 'r')
file = f.read()
soup = bs.BeautifulSoup(file,'xml')

In [36]:
metadata = soup.find('Metadata')

In [37]:
page = soup.find('Page')
print(page['imageFilename'],page['imageWidth'],page['imageHeight'])

4.jpg 2251 2870


In [38]:
table_regions = soup.findAll('TableRegion')

In [39]:
header = '''<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PcGts xmlns="http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15 http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15/pagecontent.xsd"></PcGts>'''

In [40]:
temp = table_regions[0].find('Coords')
custom = table_regions[0]['custom']

In [41]:
custom

'readingOrder {index:0;}'

### Read ICDAR XML file

In [42]:
f = open(f'{icdar}/{filename}', 'r')
file = f.read()
icdar_soup = bs.BeautifulSoup(file,'xml')

In [43]:
tables = icdar_soup.findAll('table')

In [44]:
corner = '''<CornerPts>0 1 2 3</CornerPts>'''
corner_new = bs.BeautifulSoup(corner, 'xml')

corner_pts =  corner_new.find('CornerPts')
corner_pts

<CornerPts>0 1 2 3</CornerPts>

In [45]:
soup_new = bs.BeautifulSoup(header, 'xml')
soup_new.select_one('PcGts').append(metadata)

page_point = Tag(builder=soup_new.builder, 
               name='Page', 
               attrs={'imageFilename':page['imageFilename'],'imageWidth':page['imageWidth'],'imageHeight':page['imageHeight']})

soup_new.select_one('PcGts').append(page_point)

for table in tables:
    coords =  table.find('Coords')
    table_bbox =  get_coordinates_transkribus(coords['points'])
    trans_cell = f'{table_bbox[0]},{table_bbox[1]} {table_bbox[2]},{table_bbox[1]} {table_bbox[2]},{table_bbox[3]} {table_bbox[0]},{table_bbox[3]}'
    temp_copy = copy.copy(temp) 
    temp_copy['points'] = trans_cell
#     print(table['id'],coords['points'],table_bbox,trans_cell)
    
    in_point = Tag(builder=soup_new.builder, 
               name='TableRegion', 
               attrs={'id':table['id'],'custom':custom})
    in_point.append(temp_copy)
    
    cells = table.findAll('cell')
    count = len(cells)
    print(f'Table: {trans_cell} #cells:{count}')
    #Cell addition
    for cell in cells:
        rowspan = abs(int(cell['start-row'])-int(cell['end-row']))
        colspan = abs(int(cell['start-col'])-int(cell['end-col']))
        print(rowspan,colspan)
        if cell.has_attr('header'):
            print(2,'---',cell)
            if cell['header'] == 'true':
                cell_point = Tag(builder=soup_new.builder, name='TableCell', 
                             attrs={'row':cell['start-row'], 'col':cell['start-col'], 'id': cell['id'], 'rowSpan':rowspan, 'colSpan':colspan, 'type':"header", 'custom':'''structure {type:header;}'''})
            else:
                cell_point = Tag(builder=soup_new.builder, name='TableCell', 
                             attrs={'row':cell['start-row'], 'col':cell['start-col'], 'id': cell['id'], 'rowSpan':rowspan, 'colSpan':colspan})
        else:
            print(1,'---',cell)
            cell_point = Tag(builder=soup_new.builder, name='TableCell', 
                         attrs={'row':cell['start-row'], 'col':cell['start-col'], 'id': cell['id'], 'rowSpan':rowspan, 'colSpan':colspan})
            
        cell_bbox = cell.find('Coords')
        cell_bbox = get_coordinates_transkribus(cell_bbox['points'])
        trans_cell = f'{cell_bbox[0]},{cell_bbox[1]} {cell_bbox[0]},{cell_bbox[3]} {cell_bbox[2]},{cell_bbox[3]} {cell_bbox[2]},{cell_bbox[1]}'
#         trans_cell = f'{cell_bbox[0]},{cell_bbox[1]} {cell_bbox[0]},{cell_bbox[3]} {cell_bbox[2]},{cell_bbox[3]} {cell_bbox[2]},{cell_bbox[1]}'
        temp_copy = copy.copy(temp) 
        temp_copy['points'] = trans_cell
        cell_point.append(temp_copy)
        temp_copy = copy.copy(corner_pts) 
        cell_point.append(temp_copy)
        in_point.append(cell_point) 
    soup_new.select_one('Page').append(in_point)

Table: 314,241 2188,241 2188,1017 314,1017 #cells:207
1 12
2 --- <cell end-col="12" end-row="1" header="true" id="TableCell_1598882095265_10225" start-col="0" start-row="0"><Coords points="314,241 314,340 2188,241 2188,340"/></cell>
2 1
2 --- <cell end-col="1" end-row="3" header="true" id="TableCell_1598882109286_10304" start-col="0" start-row="1"><Coords points="314,340 314,412 738,340 738,412"/></cell>
1 2
2 --- <cell end-col="3" end-row="2" header="true" id="TableCell_1598882113658_10309" start-col="1" start-row="1"><Coords points="738,340 738,379 920,340 920,379"/></cell>
2 1
2 --- <cell end-col="4" end-row="3" header="true" id="TableCell_1598882116419_10315" start-col="3" start-row="1"><Coords points="920,340 920,412 1063,340 1063,412"/></cell>
2 1
2 --- <cell end-col="5" end-row="3" header="true" id="TableCell_1598882118425_10320" start-col="4" start-row="1"><Coords points="1063,340 1063,412 1174,340 1174,412"/></cell>
2 1
2 --- <cell end-col="6" end-row="3" header="true" id="Tab

In [46]:
with open(f"{output_path}/{filename}", "w") as file:
    file.write(str(soup_new))

In [29]:
print(f"{output_path}/{filename}")

/data/glosat/Code-Git/docExtractor-master/glosat/Data_conversion/Fine/ICDAR_Transkribus/1.xml


In [20]:
cell = cells[0]
cell.has_attr('header')

True

In [21]:
table_bbox = get_coordinates_transkribus("331,248 331,285 589,285 589,248")
trans_cell = f'{table_bbox[0]},{table_bbox[1]} {table_bbox[0]},{table_bbox[3]} {table_bbox[2]},{table_bbox[3]} {table_bbox[2]},{table_bbox[1]}'
# trans_cell = f'{table_bbox[0]},{table_bbox[1]} {table_bbox[2]},{table_bbox[1]} {table_bbox[2]},{table_bbox[3]} {table_bbox[0]},{table_bbox[3]}'

print(trans_cell,table_bbox)

331,248 331,285 589,285 589,248 [331, 248, 589, 285]


In [75]:
soup_new

<?xml version="1.0" encoding="utf-8"?>
<PcGts xmlns="http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15 http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15/pagecontent.xsd"><Metadata>
<Creator>Transkribus</Creator>
<Created>2020-08-31T15:22:00.102+02:00</Created>
<LastChange>2020-08-31T15:31:28.777+02:00</LastChange>
</Metadata><Page imageFilename="1.jpg" imageHeight="2870" imageWidth="2220"><TableRegion custom="readingOrder {index:0;}" id="Table_1598880582880_6154"><Coords points="331,175 331,642 2163,642 2163,175"/><TableCell col="0" colSpan="16" id="TableCell_1598880639599_6348" row="0" rowSpan="1"><Coords points="331,175 331,248 2163,248 2163,175"/></TableCell><TableCell col="0" colSpan="1" id="TableCell_1598880654365_6364" row="1" rowSpan="1"><Coords points="331,248 331,285 589,285 589,248"/></TableCell><TableCell col="

In [63]:
cell = cells[0]

rowspan = abs(int(cell['start-row'])-int(cell['end-row']))
colspan = abs(int(cell['start-col'])-int(cell['end-col']))
print(rowspan,colspan)

1 16


In [58]:
f'<cell end-col="1" end-row="2" header="true" id="TableCell_1598880654365_6364" start-col="0" start-row="1">'

'<cell end-col="1" end-row="2" header="true" id="TableCell_1598880654365_6364" start-col="0" start-row="1">'

In [36]:
soup_new = bs.BeautifulSoup(header, 'xml')
soup_new.select_one('PcGts').append(metadata)

page_point = Tag(builder=soup_new.builder, 
               name='Page', 
               attrs={'imageFilename':page['imageFilename'],'imageWidth':page['imageWidth'],'imageHeight':page['imageHeight']})

soup_new.select_one('PcGts').append(page_point)
for table in table_regions:
    temp = table.find('Coords')
    table_bbox =  get_coordinates_transkribus(temp['points'])

    ### Add table cells from ICDAR using the table_bbox id###     id = f'{table_bbox[0]}_{table_bbox[1]}_{table_bbox[2]}_{table_bbox[3]}'
    for idx in range(len(icdar_parsed)):
        overlap_score = tsa.how_much_contained(table_bbox,icdar_parsed[idx]['region'])
        overlap_score2 = tsa.how_much_contained(icdar_parsed[idx]['region'],table_bbox)
        print(idx,table_bbox,icdar_parsed[idx]['region'], overlap_score, overlap_score2)
        if overlap_score>0.9 or overlap_score2>0.9:
            in_point = Tag(builder=soup_new.builder, 
               name='TableRegion', 
               attrs={'id':table['id'],'custom':table['custom']})
            in_point.append(temp)
            for cell in icdar_parsed[idx]['cells']:
                cell_point = Tag(builder=soup_new.builder, name='TableCell', 
                                 attrs={'row':"1", 'col':"0", 'rowSpan':"1", 'colSpan':"1"})
                cell_bbox = [int(i) for i in cell]
                trans_cell = f'{cell[0]},{cell[1]} {cell[0]},{cell[3]} {cell[2]},{cell[3]} {cell[2]},{cell[0]}'
                temp_copy = copy.copy(temp) 
                temp_copy['points'] = trans_cell
                cell_point.append(temp_copy)
                in_point.append(cell_point) 
                del cell_point

            soup_new.select_one('Page').append(in_point)

NameError: name 'icdar_parsed' is not defined

In [17]:
with open(f"{output_path}/{filename}", "w") as file:
    file.write(str(soup_new))

In [18]:
for idx in range(len(icdar_parsed)):
    print(icdar_parsed[idx]['region'])

(314.0, 241.0, 2188.0, 1017.0)
(315.0, 1010.0, 2199.0, 1653.0)
(1478.0, 1953.0, 2174.0, 2446.0)
(317.0, 1953.0, 1478.0, 2446.0)


In [19]:
for table in table_regions:
    temp = table.find('Coords')
    table_bbox =  get_coordinates_transkribus(temp['points'])
    print(table_bbox)

[314, 241, 2188, 340]
[315, 1010, 2199, 1160]
[317, 1953, 1478, 2061]
[315, 149, 2189, 244]


In [20]:
output_path

'/data/glosat/Code-Git/docExtractor-master/glosat/Data_conversion/Fine/ICDAR_Transkribus'

### Direct from ICDAR

In [35]:
soup_new = bs.BeautifulSoup(header, 'xml')
soup_new.select_one('PcGts').append(metadata)

page_point = Tag(builder=soup_new.builder, 
               name='Page', 
               attrs={'imageFilename':page['imageFilename'],'imageWidth':page['imageWidth'],'imageHeight':page['imageHeight']})

soup_new.select_one('PcGts').append(page_point)

### Add table cells from ICDAR using the table_bbox id###     id = f'{table_bbox[0]}_{table_bbox[1]}_{table_bbox[2]}_{table_bbox[3]}'
for idx in range(len(icdar_parsed)):
    cell = icdar_parsed[idx]['region']  #Table cell
    cell = [int(i) for i in cell]
    trans_cell = f'{cell[0]},{cell[1]} {cell[0]},{cell[3]} {cell[2]},{cell[3]} {cell[2]},{cell[1]}'
    temp_copy = copy.copy(temp) 
    temp_copy['points'] = trans_cell
    
    in_point = Tag(builder=soup_new.builder, 
                   name='TableRegion', 
                   attrs={'id':idx,'custom':custom})
    in_point.append(temp_copy)
    
    count = len(icdar_parsed[idx]['cells'])
    print(f'Table: {trans_cell} #cells:{count}')
    
    #Cell addition
    for cell in icdar_parsed[idx]['cells']:
        cell_point = Tag(builder=soup_new.builder, name='TableCell', 
                         attrs={'row':"1", 'col':"0", 'rowSpan':"1", 'colSpan':"1", 'type':"header"})
        cell = [int(i) for i in cell]
        trans_cell = f'{cell[0]},{cell[1]} {cell[0]},{cell[3]} {cell[2]},{cell[3]} {cell[2]},{cell[1]}'
        temp_copy = copy.copy(temp) 
        temp_copy['points'] = trans_cell
        cell_point.append(temp_copy)
        in_point.append(cell_point) 
        del cell_point

    soup_new.select_one('Page').append(in_point)

Table: 314,241 314,1017 2188,1017 2188,241 #cells:207
Table: 315,1010 315,1653 2199,1653 2199,1010 #cells:113
Table: 1478,1953 1478,2446 2174,2446 2174,1953 #cells:21
Table: 317,1953 317,2446 1478,2446 1478,1953 #cells:17


In [36]:
with open(f"{output_path}/{filename}", "w") as file:
    file.write(str(soup_new.prettify()))

In [32]:
len(icdar_parsed[idx]['cells'])

17

In [37]:
soup_new.prettify()

'<?xml version="1.0" encoding="utf-8"?>\n<PcGts xmlns="http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15 http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15/pagecontent.xsd">\n <Metadata>\n  <Creator>\n   Transkribus\n  </Creator>\n  <Created>\n   2020-08-31T15:22:00.262+02:00\n  </Created>\n  <LastChange>\n   2020-09-14T13:37:32.869+02:00\n  </LastChange>\n </Metadata>\n <Page imageFilename="4.jpg" imageHeight="2870" imageWidth="2251">\n  <TableRegion custom="readingOrder {index:0;}" id="0">\n   <Coords points="314,241 314,1017 2188,1017 2188,241"/>\n   <TableCell col="0" colSpan="1" row="1" rowSpan="1" type="header">\n    <Coords points="314,241 314,340 2188,340 2188,241"/>\n   </TableCell>\n   <TableCell col="0" colSpan="1" row="1" rowSpan="1" type="header">\n    <Coords points="314,340 314,412 738,412 738,340"/>\n   </T